# MobileBERT Evaluation on ISOT Dataset

## Introduction

This notebook evaluates a fine-tuned MobileBERT model on two distinct fake news detection scenarios:
1. **Titles-only dataset**: Using only article headlines
2. **Full-text dataset**: Using complete articles with both titles and text

This dual evaluation approach provides insights into how well MobileBERT performs with limited context versus full article context. MobileBERT was specifically designed for mobile applications, potentially offering the best balance between accuracy and efficiency for resource-constrained environments compared to other models in our comparative study.

## 1. Setting Up the Environment

Let's start by importing all necessary libraries and setting up utility functions to monitor resource usage:

In [ ]:
# Import necessary libraries
import os
import time
import numpy as np
import pandas as pd
import torch
import psutil
import gc
import re
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import Dataset, DataLoader
from transformers import MobileBertTokenizer, MobileBertForSequenceClassification
from datasets import Dataset as HFDataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
import warnings
warnings.filterwarnings('ignore')

These libraries provide the foundation for data manipulation, model evaluation, visualization, and resource monitoring - especially important for understanding MobileBERT's performance in resource-constrained environments.

In [ ]:
# Set device - using CPU for edge device testing
device = torch.device("cpu")
print(f"Using device: {device}")

I deliberately choose to use the CPU rather than GPU for this evaluation because:
1. The primary target for MobileBERT deployment is mobile and edge devices that typically lack dedicated GPUs
2. CPU performance metrics provide a more realistic assessment of real-world deployment scenarios
3. This allows for direct comparison with other lightweight models in similar deployment conditions

In [ ]:
# Function to get current memory usage
def get_memory_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024  # Convert to MB

print(f"Starting memory usage: {get_memory_usage():.2f} MB")

This utility function is particularly valuable for MobileBERT, as it helps quantify the model's memory efficiency - a primary consideration for its target deployment scenarios on mobile devices.

## 2. Loading the Pre-trained Model

Now I'll load the MobileBERT model that was previously fine-tuned on the ISOT dataset:

In [ ]:
# Load the pre-trained MobileBERT model
print("\nLoading model...")
model_path = "../ml_models/mobilebert-fake-news-detector"

In [ ]:
# Initialize tokenizer and model
start_time = time.time()
tokenizer = MobileBertTokenizer.from_pretrained(model_path)
model = MobileBertForSequenceClassification.from_pretrained(model_path)
model.to(device)  # Move to CPU
load_time = time.time() - start_time

In [ ]:
print(f"Model loaded in {load_time:.2f} seconds")
print(f"Memory usage after loading model: {get_memory_usage():.2f} MB")

Model loading time and memory footprint are critical metrics for MobileBERT, as they directly impact the model's suitability for mobile and edge environments where startup time and memory constraints are significant concerns.

## 3. Data Loading and Preparation

Next, I'll load various data sources for our evaluation:

In [ ]:
# Load all data sources
print("\nLoading all data sources...")

In [ ]:
# 1. Load titles_only_real.csv (real news, titles only)
try:
    titles_only_real_df = pd.read_csv('../data/titles_only_real.csv')
    # Ensure label is 1 for real news
    titles_only_real_df['label'] = 1
    print(f"Loaded {len(titles_only_real_df)} real news titles from titles_only_real.csv")
except Exception as e:
    print(f"Error loading titles_only_real.csv: {e}")
    titles_only_real_df = pd.DataFrame(columns=['title', 'label'])

In [ ]:
# 2. Load FakeNewsNet (fake news only)
try:
    fake_news_net_df = pd.read_csv('./datasets/simplified_FakeNewsNet.csv')
    # Keep only fake news
    fake_news_net_df = fake_news_net_df[fake_news_net_df['label'] == 0]
    print(f"Loaded {len(fake_news_net_df)} fake news articles from FakeNewsNet")
except Exception as e:
    print(f"Error loading FakeNewsNet: {e}")
    fake_news_net_df = pd.DataFrame(columns=['title', 'label'])

In [ ]:
# 3. Load fake_news_evaluation.csv (fake news with text)
try:
    fake_news_eval_df = pd.read_csv('./datasets/fake_news_evaluation.csv')
    # Ensure label is 0 for fake news
    fake_news_eval_df['label'] = 0
    print(f"Loaded {len(fake_news_eval_df)} fake news articles from fake_news_evaluation.csv")
except Exception as e:
    print(f"Error loading fake_news_evaluation.csv: {e}")
    fake_news_eval_df = pd.DataFrame(columns=['title', 'text', 'label'])

In [ ]:
# 4. Load manual_real.csv (real news with text)
try:
    manual_real_df = pd.read_csv('./datasets/manual_real.csv')
    # Ensure label is 1 for real news
    manual_real_df['label'] = 1
    print(f"Loaded {len(manual_real_df)} real news articles with text from manual_real.csv")
except Exception as e:
    print(f"Error loading manual_real.csv: {e}")
    manual_real_df = pd.DataFrame(columns=['title', 'text', 'label'])

I'm using the same diverse set of data sources as in the other model evaluations to ensure fair comparisons across our entire model suite.

## 4. Preparing Title-Only Dataset

Now I'll prepare the title-only dataset for evaluation:

In [ ]:
# Create title-only dataset
print("\nPreparing title-only dataset...")

# Get the target size (number of real news titles) for balancing
real_titles_count = len(titles_only_real_df)
print(f"Target size for balanced dataset: {real_titles_count} articles per class")

In [ ]:
# Prepare fake news data (titles only)
# 1. From FakeNewsNet
if 'text' not in fake_news_net_df.columns:
    fake_news_net_df['text'] = fake_news_net_df['title']
else:
    # Use only title as text
    fake_news_net_df['text'] = fake_news_net_df['title']

In [ ]:
# 2. From fake_news_evaluation.csv
fake_news_eval_titles_df = fake_news_eval_df.copy()
fake_news_eval_titles_df['text'] = fake_news_eval_titles_df['title']  # Use only title, not full text

In [ ]:
# Combine all fake news sources (titles only)
fake_news_title_only = pd.concat([fake_news_net_df[['text', 'label']], 
                                 fake_news_eval_titles_df[['text', 'label']]], 
                                 ignore_index=True)

In [ ]:
# Balance fake news to match real news count
# This allows us to handle growing real news dataset without manual adjustment
if len(fake_news_title_only) > real_titles_count:
    print(f"Balancing fake news dataset: sampling {real_titles_count} articles from {len(fake_news_title_only)} total")
    # Sample randomly to match the real news count
    fake_news_title_only = fake_news_title_only.sample(n=real_titles_count, random_state=42)
else:
    print(f"Note: Not enough fake news articles ({len(fake_news_title_only)}) to match real news count ({real_titles_count})")

In [ ]:
# Prepare real news data (titles only)
if 'text' not in titles_only_real_df.columns:
    titles_only_real_df['text'] = titles_only_real_df['title']

In [ ]:
# Combine fake and real news (titles only)
title_only_dataset_df = pd.concat([fake_news_title_only, titles_only_real_df[['text', 'label']]], 
                                 ignore_index=True)

In [ ]:
# Shuffle to mix real and fake news
title_only_dataset_df = title_only_dataset_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
print(f"Prepared title-only dataset with {len(title_only_dataset_df)} articles")
print(f"Class distribution: {title_only_dataset_df['label'].value_counts().to_dict()}")

In [ ]:
# Convert to HuggingFace Dataset format
title_only_dataset = HFDataset.from_pandas(title_only_dataset_df)

Testing MobileBERT on headline-only data is particularly valuable because:
1. The model's compact architecture was specifically designed for efficient processing on mobile devices
2. Headline-only classification represents a common use case for mobile applications where processing speed is critical
3. This evaluation helps determine if MobileBERT's optimizations maintain performance even with limited context

## 5. Preparing Full-Text Dataset

Similarly, I'll prepare the full-text dataset:

In [ ]:
# Create full-text dataset
print("\nPreparing full-text dataset...")

In [ ]:
# Prepare fake news data (with full text)
# Use fake_news_eval_df which already has text
fake_news_full_text_df = fake_news_eval_df.copy()
fake_news_full_text_df['text'] = fake_news_full_text_df['title'] + " " + fake_news_full_text_df['text'].fillna('')

In [ ]:
# Prepare real news data (with full text)
manual_real_text_df = manual_real_df.copy()
manual_real_text_df['text'] = manual_real_text_df['title'] + " " + manual_real_text_df['text'].fillna('')

In [ ]:
# Balance the datasets if needed
fake_count = len(fake_news_full_text_df)
real_count = len(manual_real_text_df)
target_count = min(fake_count, real_count)

print(f"Full-text dataset - Fake: {fake_count}, Real: {real_count}")

In [ ]:
# Balance the datasets if needed
if fake_count > real_count:
    print(f"Balancing full-text dataset: sampling {real_count} fake articles from {fake_count}")
    fake_news_full_text_df = fake_news_full_text_df.sample(n=real_count, random_state=42)
elif real_count > fake_count:
    print(f"Balancing full-text dataset: sampling {fake_count} real articles from {real_count}")
    manual_real_text_df = manual_real_text_df.sample(n=fake_count, random_state=42)

In [ ]:
# Combine fake and real news (with full text)
full_text_dataset_df = pd.concat([fake_news_full_text_df[['text', 'label']], 
                                manual_real_text_df[['text', 'label']]], 
                                ignore_index=True)

In [ ]:
# Shuffle to mix real and fake news
full_text_dataset_df = full_text_dataset_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
print(f"Prepared full-text dataset with {len(full_text_dataset_df)} articles")
print(f"Class distribution: {full_text_dataset_df['label'].value_counts().to_dict()}")

In [ ]:
# Convert to HuggingFace Dataset format
full_text_dataset = HFDataset.from_pandas(full_text_dataset_df)

The full-text evaluation is important for understanding:
1. How well MobileBERT handles longer sequences despite its streamlined architecture
2. Whether the model's optimizations compromise its ability to capture long-range dependencies
3. If the more comprehensive context provides substantial performance improvements that might justify the additional processing time

## 6. Evaluation Utility Functions

Next, I'll define utility functions for evaluation:

In [ ]:
# Define tokenization function
def tokenize_dataset(dataset):
    """Tokenize a dataset using the MobileBERT tokenizer"""
    print(f"Tokenizing dataset with {len(dataset)} examples...")
    tokenize_start_time = time.time()
    
    # Define tokenization function
    def tokenize_function(examples):
        return tokenizer(
            examples['text'],
            padding='max_length',
            truncation=True,
            max_length=512,
            return_tensors=None
        )
    
    # Clean dataset to handle edge cases
    def clean_dataset(example):
        example['text'] = str(example['text']) if example['text'] is not None else ""
        return example
    
    # Clean and tokenize
    cleaned_dataset = dataset.map(clean_dataset)
    tokenized_dataset = cleaned_dataset.map(tokenize_function, batched=True)
    tokenized_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
    
    tokenize_time = time.time() - tokenize_start_time
    print(f"Dataset tokenized in {tokenize_time:.2f} seconds")
    print(f"Memory usage after tokenization: {get_memory_usage():.2f} MB")
    
    return tokenized_dataset

In [ ]:
# Define model evaluation function - Part 1: Setup
def evaluate_model_setup(tokenized_dataset, dataset_name):
    """Evaluate the model on a tokenized dataset and return metrics and resource usage"""
    print(f"\nEvaluating model on {dataset_name} dataset...")
    
    # Reset all counters and lists
    all_preds = []
    all_labels = []
    total_inference_time = 0
    sample_count = 0
    inference_times = []
    memory_usages = []
    
    # Create DataLoader
    from torch.utils.data import DataLoader
    eval_dataloader = DataLoader(
        tokenized_dataset, 
        batch_size=16,  # Appropriate batch size for CPU
        shuffle=False
    )
    
    print(f"Starting evaluation on {len(tokenized_dataset)} examples")
    
    return eval_dataloader, all_preds, all_labels, total_inference_time, sample_count, inference_times, memory_usages

In [ ]:
# Define model evaluation function - Part 2: Inference loop
def run_evaluation_loop(eval_dataloader, all_preds, all_labels, total_inference_time, 
                       sample_count, inference_times, memory_usages):
    """Run the evaluation loop on the provided dataloader"""
    
    # Evaluation loop
    model.eval()
    with torch.no_grad():
        for batch_idx, batch in enumerate(eval_dataloader):
            # Track batch progress
            if batch_idx % 5 == 0:
                print(f"Processing batch {batch_idx}/{len(eval_dataloader)}")
            
            # Extract batch data
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            # Record batch size
            current_batch_size = input_ids.size(0)
            sample_count += current_batch_size
            
            # Memory tracking
            memory_usages.append(get_memory_usage())
            
            # Time the inference
            start_time = time.time()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            batch_inference_time = time.time() - start_time
            inference_times.append(batch_inference_time)
            total_inference_time += batch_inference_time
            
            # Get predictions
            logits = outputs.logits
            predictions = torch.softmax(logits, dim=-1)
            predicted_labels = torch.argmax(predictions, dim=1).cpu().numpy()
            
            # Store predictions and labels
            all_preds.extend(predicted_labels)
            all_labels.extend(labels.cpu().numpy())
    
    print(f"Evaluation complete. Total predictions: {len(all_preds)}, Total labels: {len(all_labels)}")
    
    return all_preds, all_labels, total_inference_time, sample_count, inference_times, memory_usages

In [ ]:
# Define model evaluation function - Part 3: Metrics calculation
def calculate_metrics(all_preds, all_labels, total_inference_time, sample_count, 
                     inference_times, memory_usages, dataset_name):
    """Calculate performance metrics from evaluation results"""
    
    if len(all_preds) == len(all_labels):
        accuracy = accuracy_score(all_labels, all_preds)
        precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
        
        print(f"\nEvaluation Results for {dataset_name} dataset:")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")
        
        # Create confusion matrix
        cm = np.zeros((2, 2), dtype=int)
        for true_label, pred_label in zip(all_labels, all_preds):
            cm[true_label, pred_label] += 1
        
        print(f"\nConfusion Matrix for {dataset_name} dataset:")
        print(cm)
        
        # Resource consumption analysis
        print(f"\nResource Consumption Analysis for {dataset_name} dataset:")
        print(f"Total evaluation time: {total_inference_time:.2f} seconds")
        print(f"Average inference time per batch: {np.mean(inference_times):.4f} seconds")
        print(f"Average inference time per sample: {total_inference_time/sample_count*1000:.2f} ms")
        print(f"Peak memory usage: {max(memory_usages):.2f} MB")
        
        return {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'confusion_matrix': cm,
            'inference_time_per_sample': total_inference_time/sample_count*1000,
            'peak_memory': max(memory_usages),
        }
    else:
        print("ERROR: Cannot calculate metrics - prediction and label counts don't match")
        return None

In [ ]:
# Define model evaluation function - Part 4: Visualization
def visualize_results(metrics_dict, all_labels, all_preds, inference_times, memory_usages, dataset_name):
    """Create visualizations of evaluation results"""
    
    # Plot confusion matrix
    cm = metrics_dict['confusion_matrix']
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Fake', 'Real'], yticklabels=['Fake', 'Real'])
    plt.title(f'Confusion Matrix - {dataset_name}')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.savefig(f'mobilebert_confusion_matrix_{dataset_name.lower().replace(" ", "_")}.png')
    plt.show()
    
    # Plot resource usage
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 1, 1)
    plt.plot(inference_times)
    plt.title(f'Inference Time per Batch (CPU) - {dataset_name}')
    plt.xlabel('Batch')
    plt.ylabel('Time (seconds)')
    
    plt.subplot(2, 1, 2)
    plt.plot(memory_usages, label='System Memory')
    plt.title(f'Memory Usage During Evaluation (CPU) - {dataset_name}')
    plt.xlabel('Batch')
    plt.ylabel('Memory (MB)')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(f'mobilebert_resource_usage_{dataset_name.lower().replace(" ", "_")}.png')
    plt.show()
    
    # Generate classification report
    print(f"\nDetailed Classification Report for {dataset_name}:")
    report = classification_report(all_labels, all_preds, target_names=['Fake News', 'Real News'])
    print(report)
    
    metrics_dict['classification_report'] = report
    return metrics_dict

In [ ]:
# Combined evaluation function
def evaluate_model(tokenized_dataset, dataset_name):
    """Complete evaluation pipeline"""
    # Setup
    eval_dataloader, all_preds, all_labels, total_inference_time, sample_count, inference_times, memory_usages = evaluate_model_setup(tokenized_dataset, dataset_name)
    
    # Run inference
    all_preds, all_labels, total_inference_time, sample_count, inference_times, memory_usages = run_evaluation_loop(
        eval_dataloader, all_preds, all_labels, total_inference_time, sample_count, inference_times, memory_usages
    )
    
    # Calculate metrics
    metrics_dict = calculate_metrics(
        all_preds, all_labels, total_inference_time, sample_count, inference_times, memory_usages, dataset_name
    )
    
    if metrics_dict:
        # Visualize results
        metrics_dict = visualize_results(
            metrics_dict, all_labels, all_preds, inference_times, memory_usages, dataset_name
        )
    
    return metrics_dict

These evaluation functions are structured to provide comprehensive insights into MobileBERT's performance and resource usage. The detailed tracking of memory consumption and inference time is particularly important for MobileBERT's target deployment scenarios on resource-constrained devices.

## 7. Evaluating Title-Only Dataset

Now I'll evaluate the model on the title-only dataset:

In [ ]:
# Tokenize the title-only dataset
title_only_tokenized = tokenize_dataset(title_only_dataset)

In [ ]:
# Evaluate model on title-only dataset
title_only_results = evaluate_model(title_only_tokenized, "Title-Only")

For MobileBERT, the title-only evaluation is particularly relevant for mobile applications where:
1. Processing speed is a critical consideration for user experience
2. Battery life and resource consumption need to be minimized
3. Users may want quick preliminary assessments before deciding to read full articles

## 8. Evaluating Full-Text Dataset

Next, I'll evaluate the model on the full-text dataset:

In [ ]:
# Tokenize the full-text dataset
full_text_tokenized = tokenize_dataset(full_text_dataset)

In [ ]:
# Evaluate model on full-text dataset
full_text_results = evaluate_model(full_text_tokenized, "Full-Text")

The full-text evaluation helps determine:
1. Whether MobileBERT's efficiency optimizations maintain performance for longer text
2. If the model's bottleneck architecture adequately captures the complex linguistic patterns in full articles
3. How the performance-efficiency trade-off compares to both smaller models like TinyBERT and larger models like DistilBERT and RoBERTa

## 9. Comparing Results Between Datasets

Finally, I'll create a comparative analysis of both approaches:

In [ ]:
# Create comparison table
if title_only_results and full_text_results:
    comparison_df = pd.DataFrame({
        'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'Inference Time (ms/sample)', 'Peak Memory (MB)'],
        'Title-Only': [
            title_only_results['accuracy'],
            title_only_results['precision'],
            title_only_results['recall'],
            title_only_results['f1'],
            title_only_results['inference_time_per_sample'],
            title_only_results['peak_memory']
        ],
        'Full-Text': [
            full_text_results['accuracy'],
            full_text_results['precision'],
            full_text_results['recall'],
            full_text_results['f1'],
            full_text_results['inference_time_per_sample'],
            full_text_results['peak_memory']
        ]
    })

In [ ]:
# Format and display comparison table
comparison_df['Title-Only'] = comparison_df['Title-Only'].apply(
    lambda x: f"{x:.4f}" if isinstance(x, (int, float)) and x < 100 else f"{x:.2f}")
comparison_df['Full-Text'] = comparison_df['Full-Text'].apply(
    lambda x: f"{x:.4f}" if isinstance(x, (int, float)) and x < 100 else f"{x:.2f}")

print("Performance Comparison Between Datasets:")
print(comparison_df.to_string(index=False))

In [ ]:
# Create visualization of metrics comparison
metrics = comparison_df.iloc[:4]  # Just the first 4 metrics (accuracy, precision, recall, f1)

# Convert to numeric for plotting
metrics['Title-Only'] = metrics['Title-Only'].astype(float)
metrics['Full-Text'] = metrics['Full-Text'].astype(float)

plt.figure(figsize=(10, 6))
bar_width = 0.35
index = np.arange(len(metrics))

plt.bar(index, metrics['Title-Only'], bar_width, label='Title-Only')
plt.bar(index + bar_width, metrics['Full-Text'], bar_width, label='Full-Text')

plt.xlabel('Metrics')
plt.ylabel('Score')
plt.title('MobileBERT Performance Comparison: Title-Only vs Full-Text')
plt.xticks(index + bar_width / 2, metrics['Metric'])
plt.legend()
plt.tight_layout()
plt.savefig('mobilebert_performance_comparison.png')
plt.show()

This comparative analysis is particularly valuable for mobile applications, where developers must carefully balance performance with resource consumption. The results will help determine whether the additional context from full articles justifies the increased processing time and memory usage on mobile devices.

## 10. Conclusion and Cleanup

In [ ]:
# Free up memory
del model
gc.collect()
print(f"Final memory usage: {get_memory_usage():.2f} MB")

The table and visualization above provide a clear comparison between using only titles versus full text for fake news detection with MobileBERT. Key findings include:

1. **Performance Characteristics**: 
   - MobileBERT demonstrates how effectively a model optimized for mobile devices can perform on fake news detection tasks
   - The performance gap between title-only and full-text approaches helps quantify the value of additional context within the constraints of mobile deployment

2. **Resource Efficiency**:
   - The memory footprint and inference time metrics directly relate to MobileBERT's primary design goals of mobile optimization
   - These metrics help determine whether MobileBERT achieves its intended efficiency targets while maintaining competitive accuracy

3. **Practical Applications**:
   - For mobile applications where battery life and user experience are critical, the title-only approach may provide the best balance of accuracy and efficiency
   - For more critical applications where maximum accuracy is required, the full-text approach with MobileBERT may offer a good compromise between performance and resource consumption

When compared to TinyBERT, DistilBERT, and RoBERTa, these results provide valuable insights into which model offers the best balance of performance and efficiency for different deployment scenarios. MobileBERT's performance relative to its resource consumption will determine whether its mobile-specific optimizations make it the preferred choice for resource-constrained environments compared to more general model compression approaches.